# Requetes et visualisation de données à partir de l'API entreprise RGE
**documentation** : https://api.gouv.fr/documentation/api_professionnels_rge \
**api** : https://data.ademe.fr/data-fair/api/v1/datasets/liste-des-entreprises-rge-2/api-docs.json \
**infos ademe** : https://data.ademe.fr/datasets/liste-des-entreprises-rge-2/api-doc \



In [2]:
import requests
import pandas as pd
import folium

In [3]:
# fonction permettant de gérer les différentes requètes du notebook
def requete_api(url, params=None):
    try:
        if params is not None:
            reponse = requests.get(url, params=params)
        else:
            reponse = requests.get(url)
        # Lève une exception si la requête échoue
        reponse.raise_for_status()
        return reponse.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
        return None

## Test de l'API

Liste des champs possibles à extraire : \
https://data.ademe.fr/data-fair/api/v1/datasets/liste-des-entreprises-rge-2/

siret, nom_entreprise, adresse, code_postal, commune, latitude, longitude, telephone, email, site_internet, code_qualification, nom_qualification, url_qualification, nom_certificat, domaine, meta_domaine, organisme, particulier, lien_date_debut, lien_date_fin

### Premier test
On extraie l'ensemble des entreprise pour le code_postal=31000.\
Un tri par siret est effectué

In [15]:
url = "https://data.ademe.fr/data-fair/api/v1/datasets/liste-des-entreprises-rge-2/lines?"

params = {
    "sort": "siret",
    "highlight": "code_postal",
    "q":"31000",
}

data = requete_api(url, params=params)

if data is not None:
    df = pd.DataFrame(data)

print(df.dtypes)
# le DataFrame a trois colonnes : total (de type int64), next (de type object), et results (de type object). 

# On affiche uniquement les deux premiers éléments
print(df.results.head(2).values)

total       int64
next       object
results    object
dtype: object
[{'site_internet': '', 'lien_date_fin': '2027-09-13', 'code_postal': '31000', '_geopoint': '43.603247,1.456388', 'commune': 'TOULOUSE', 'latitude': 43.603247, 'domaine': "Fenêtres, volets, portes donnant sur l'extérieur", 'nom_entreprise': 'ETABLISSEMENTS CALFEUTRAL', 'nom_qualification': 'Fourniture et pose de menuiseries extérieures en maison individuelle, petit collectif et petit tertiaire (3511D109)', 'organisme': 'qualibat', 'telephone': '05 61 62 44 24', '_i': 105375424568186, 'url_qualification': 'https://www.qualibat.com/Views/GetFichier.aspx?fn=2024\\D31-Certificat-123893-E123893-1-20240917-RGEAnnexe.pdf', 'siret': '32012984400017', 'meta_domaine': "Travaux d'efficacité énergétique", 'code_qualification': '3511D109', '_rand': 462549, 'particulier': True, 'adresse': '11 BOULEVARD PROFESSEUR LEOPOLD ESCANDE', 'nom_certificat': 'QUALIBAT-RGE', 'email': 'info@calfeutral.com', 'lien_date_debut': '2023-09-14', 'long

### Extraction des professionnels RGE installateurs de poêles ou inserts

Pour chaque territoire, nous sortons une liste de l’ensemble des professionnels RGE installateurs de poêles ou inserts, installés sur leur territoire

on souhaite extraire toutes les entreprise du domaine : "Poêle ou insert bois" sur la commune TOULOUSE.

In [6]:
url = "https://data.ademe.fr/data-fair/api/v1/datasets/liste-des-entreprises-rge-2/lines?"

params = {
    "sort": "code_postal",
    "size": "10",
    "select": "latitude,longitude,siret,nom_entreprise,adresse,code_postal,commune,telephone,email,domaine",
    "qs": """ commune:"TOULOUSE" AND domaine:"Poêle ou insert bois" """,
}

data = requete_api(url, params=params)

if data is not None:
    df = pd.DataFrame(data)

print(df.results.values)

[{'code_postal': '31000', 'commune': 'TOULOUSE', 'latitude': 43.6, 'domaine': 'Poêle ou insert bois', 'adresse': '32 RUE MATABIAU Bâtiment A', 'nom_entreprise': 'ARTISAN LIECHTI PLOMBERIE', 'telephone': '06 31 37 52 30', 'siret': '92900954600011', 'email': 'liechtiplomberie@gmail.com', 'longitude': 1.433333, '_score': None}
 {'code_postal': '31100', 'commune': 'TOULOUSE', 'latitude': 43.516667, 'domaine': 'Poêle ou insert bois', 'adresse': '3 Rue Henri Mayer', 'nom_entreprise': 'EURL COURTOIS CHEMINEES', 'telephone': '05 62 48 15 75', 'siret': '84865962900028', 'email': 'courtois.cheminees@gmail.com', 'longitude': 1.416667, '_score': None}
 {'code_postal': '31100', 'commune': 'TOULOUSE', 'latitude': 43.552159, 'domaine': 'Poêle ou insert bois', 'adresse': '6 bis rue Henri Turner', 'nom_entreprise': 'MAGNE ET PALMER', 'telephone': '05 34 27 06 76', 'siret': '49903996400030', 'email': 'secretariat@magneetpalmer.fr', 'longitude': 1.407225, '_score': None}
 {'code_postal': '31100', 'commun

### On souhaite extraire toutes les entreprise du domaine : "Poêle ou insert bois" dans un rayon de 10km autour de Solagro.

In [7]:
# Coordonnées de Solagro
lon_solagro = 1.397389
lat_solagro = 43.602566

url = "https://data.ademe.fr/data-fair/api/v1/datasets/liste-des-entreprises-rge-2/lines?"

params = {
    "sort": "code_postal",
    "size": "100",
    "select": "latitude,longitude,siret,nom_entreprise,adresse,code_postal,commune,telephone,email,domaine",
    "qs": """ domaine:"Poêle ou insert bois" """,
    "geo_distance" : "1.397389:43.602566:10000",
}

data = requete_api(url, params=params)

if data is not None:
    df = pd.DataFrame(data)

print(df.results.values)

# On supprime les doublons
df_sans_doublons = df.drop_duplicates(subset=['results'])
print(df_sans_doublons.results.values)

[{'code_postal': '31000', 'commune': 'TOULOUSE', 'latitude': 43.6, 'domaine': 'Poêle ou insert bois', 'adresse': '32 RUE MATABIAU Bâtiment A', 'nom_entreprise': 'ARTISAN LIECHTI PLOMBERIE', 'telephone': '06 31 37 52 30', 'siret': '92900954600011', 'email': 'liechtiplomberie@gmail.com', 'longitude': 1.433333, '_score': None}
 {'code_postal': '31100', 'commune': 'TOULOUSE', 'latitude': 43.5923, 'domaine': 'Poêle ou insert bois', 'adresse': '17 impasse Teynier', 'nom_entreprise': 'ERCS', 'telephone': '05 61 31 02 47', 'siret': '41456762800048', 'email': 'larribere.ercs@gmail.com', 'longitude': 1.37658, '_score': None}
 {'code_postal': '31100', 'commune': 'TOULOUSE', 'latitude': 43.552159, 'domaine': 'Poêle ou insert bois', 'adresse': '6 bis rue Henri Turner', 'nom_entreprise': 'MAGNE ET PALMER', 'telephone': '05 34 27 06 76', 'siret': '49903996400030', 'email': 'secretariat@magneetpalmer.fr', 'longitude': 1.407225, '_score': None}
 {'code_postal': '31100', 'commune': 'TOULOUSE', 'latitude

### Visualisation des entreprises 100km autour de Solagro

In [8]:
# Créer une carte
f = folium.Figure(width=800, height=800)
m = folium.Map(location=[43.602566, 1.397389], zoom_start=12).add_to(f)

# Ajouter SOLAGRO à la carte
folium.Marker([43.602566, 1.397389], popup='SOLAGRO', icon=folium.Icon(color='red')).add_to(m)

lon = df.results.apply(lambda x: x['longitude']).values
lat = df.results.apply(lambda x: x['latitude']).values
nom_entreprise = df.results.apply(lambda x: x['nom_entreprise']).values

for i in range(len(lon)):
    folium.Marker(location=[lat[i], lon[i]], popup=nom_entreprise[i]).add_to(m)

display(m)

### Écriture dans un fichier Excel

In [13]:
# Transformation de la colonne 'results' en DataFrame
results_df = pd.json_normalize(df_sans_doublons['results'])

# Spécifier l'ordre des colonnes
ordre_colonnes = ['domaine', 'code_postal', 'commune', 'adresse', 'siret', 'nom_entreprise', 'telephone', 'email']

# Écrire dans un fichier Excel avec les colonnes dans un ordre spécifique
results_df.to_excel('Extraction_RGE_poele_insert_bois.xlsx',
                    columns=ordre_colonnes,
                    index=False)

print(output_file)


Extraction_RGE_poele_insert_bois.xlsx


## Utilisation de l'API Découpage administratif > EPCI
lien : https://geo.api.gouv.fr/decoupage-administratif/epcis

On récupère le code EPCI de Toulouse

In [38]:
url = "https://geo.api.gouv.fr/epcis?"

params = {
    "nom": "NANTES",
    "fields": "departement",
}

data = requete_api(url, params=params)

if data is not None:
    df = pd.DataFrame(data)

print(df)

                nom       code    _score
0  Nantes Métropole  244400404  0.870079


On récupère la liste des codes communes de cet EPCI

In [39]:
url = "https://geo.api.gouv.fr/epcis/244400404/communes?"

data = requete_api(url, params=None)

if data is not None:
    df = pd.DataFrame(data)

print(df.codesPostaux)

0                          [44115]
1                          [44830]
2                          [44340]
3                          [44830]
4                          [44470]
5                          [44240]
6                          [44220]
7                          [44610]
8                          [44470]
9                          [44620]
10    [44000, 44100, 44200, 44300]
11                         [44700]
12                         [44640]
13                         [44400]
14                         [44860]
15                         [44800]
16                         [44640]
17                         [44710]
18                         [44980]
19                         [44230]
20                         [44880]
21                         [44840]
22                         [44470]
23                         [44120]
Name: codesPostaux, dtype: object


On soumet une liste de code postal à l'API:
est ce qu'on doit faire 36 requetes ? ou on peut en faire une unique à partir d'une liste ??

In [40]:
url = "https://geo.api.gouv.fr/epcis/244400404/communes?"

data = requete_api(url, params=None)

if data is not None:
    df = pd.DataFrame(data)

df_result = pd.DataFrame()

for codes in df.codesPostaux:
    for code in codes:

        url = "https://data.ademe.fr/data-fair/api/v1/datasets/liste-des-entreprises-rge-2/lines?"

        params = {
            "sort": "commune",
            "size": "100",
            "select": "latitude,longitude,siret,nom_entreprise,adresse,code_postal,commune,telephone,email,domaine",
            "qs": f""" code_postal:"{code}" AND domaine:"Poêle ou insert bois" """,
        }

        data = requete_api(url, params=params)

        if data is not None:
            df = pd.DataFrame(data)

        # Concaténation des DataFrames
        df_result = pd.concat([df_result, df], ignore_index=True)

print(df_result)
    

    total                                            results
0       2  {'code_postal': '44115', 'commune': 'BASSE GOU...
1       2  {'code_postal': '44115', 'commune': 'BASSE GOU...
2       2  {'code_postal': '44340', 'commune': 'BOUGUENAI...
3       2  {'code_postal': '44340', 'commune': 'BOUGUENAI...
4       3  {'code_postal': '44470', 'commune': 'CARQUEFOU...
5       3  {'code_postal': '44470', 'commune': 'CARQUEFOU...
6       3  {'code_postal': '44470', 'commune': 'THOUARE S...
7       3  {'code_postal': '44240', 'commune': 'LA CHAPEL...
8       3  {'code_postal': '44240', 'commune': 'LA CHAPEL...
9       3  {'code_postal': '44240', 'commune': 'LA CHAPEL...
10      3  {'code_postal': '44220', 'commune': 'COUERON',...
11      3  {'code_postal': '44220', 'commune': 'COUERON',...
12      3  {'code_postal': '44220', 'commune': 'COUERON',...
13      3  {'code_postal': '44470', 'commune': 'CARQUEFOU...
14      3  {'code_postal': '44470', 'commune': 'CARQUEFOU...
15      3  {'code_postal

In [44]:
# Créer une carte
f = folium.Figure(width=800, height=800)
m = folium.Map(location=[47.21575592285178, -1.5492888343871223], zoom_start=11).add_to(f)

lon = df_result.results.apply(lambda x: x['longitude']).values
lat = df_result.results.apply(lambda x: x['latitude']).values
nom_entreprise = df_result.results.apply(lambda x: x['nom_entreprise']).values

for i in range(len(lon)):
    folium.Marker(location=[lat[i], lon[i]], popup=nom_entreprise[i]).add_to(m)

# Sauvegarder la carte en fichier HTML
m.save("carte_extraction_RGE_poele_insert_bois_NANTES.html")

display(m)

## Extrtaction des entreprises Poele ou insert bois sur France entière

In [4]:
url = "https://data.ademe.fr/data-fair/api/v1/datasets/liste-des-entreprises-rge-2/lines?"

params = {
    "sort": "code_postal",
    "size": "10000",
    "select": "siret,nom_entreprise,adresse,code_postal,commune,latitude,longitude,telephone,email,site_internet,nom_qualification,domaine,particulier",
    "qs": """ domaine:"Poêle ou insert bois" """,
}

data = requete_api(url, params=params)

if data is not None:
    df = pd.DataFrame(data)

print(df.results.size)

# On supprime les doublons
df_sans_doublons = df.drop_duplicates(subset=['results'])
print(df_sans_doublons.results.values)

# Transformation de la colonne 'results' en DataFrame
results_df = pd.json_normalize(df_sans_doublons['results'])

# Spécifier l'ordre des colonnes
ordre_colonnes = ["siret","nom_entreprise","adresse","code_postal","commune","latitude","longitude","telephone","email","site_internet","nom_qualification","domaine","particulier"]

# Écrire dans un fichier Excel avec les colonnes dans un ordre spécifique
results_df.to_excel('Extraction_RGE_poele_insert_bois_FranceEntiere.xlsx',
                    columns=ordre_colonnes,
                    index=False)



8835
[{'site_internet': '', 'code_postal': '00000', 'commune': 'COMINES', 'latitude': 50.777779, 'domaine': 'Poêle ou insert bois', 'nom_entreprise': 'PLANTEFEVE MEDHI', 'nom_qualification': "Installation d'appareil de chauffage bois indépendant : poêle et insert (5221D104)", 'telephone': '', 'siret': '00000000147220', 'particulier': True, 'adresse': 'AVENUE DES CHATEAUX 113', 'email': 'plantefevemedhi@hotmail.fr', 'longitude': 3.009033, '_score': None}
 {'site_internet': '', 'code_postal': '00000', 'commune': 'BEAURAING', 'latitude': 50.110282, 'domaine': 'Poêle ou insert bois', 'nom_entreprise': 'ROCHETTE ET FILS', 'nom_qualification': "Installation d'appareil de chauffage bois indépendant : poêle et insert (5221D104)", 'telephone': '', 'siret': '00000000147083', 'particulier': True, 'adresse': 'RUE DE ROCHEFORT, 35', 'email': 'info@rochettesprl.be', 'longitude': 4.959379, '_score': None}
 {'site_internet': '', 'code_postal': '00000', 'commune': 'BEAURAING', 'latitude': 50.110282, 'd